In [1]:
#import necessary packages
import re
import textwrap


In [41]:
def logic(output, *rules):
    #Matching rule for input text to know which propositional logic it matches
    return (output, [propositional_logic(i) + '$' for i in rules])


In [42]:
def propositional_logic(i):
    #match the selected keyword with the selected rule
    return re.sub('{(.)}', r'(?P<\1>.+?)', i)

In [43]:
def word(w):
    #returning the output as words not as letters
    return r'\b' + w + r'\b' 

In [44]:

logic('{P} ⇒ {Q}', 'if {P} then {Q}', 'if {P}, {Q}'),


(('{P} ⇒ {Q}',
  ['if (?P<P>.+?) then (?P<Q>.+?)$', 'if (?P<P>.+?), (?P<Q>.+?)$']),)

In [10]:
rules = [
    logic('{P} ⇒ {Q}',         'if {P} then {Q}', 'if {P}, {Q}'),
    logic('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}'),
    logic('{P} ⋀ {Q}',          'both {P} and {Q}'),
    logic('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    logic('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'), 
    logic('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    logic('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}', 
                               '{P} implies {Q}', '{P} therefore {Q}', 
                               '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    logic('{P} ⋀ {Q}',          '{P} and {Q}', '{P} but {Q}'),
    logic('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    ]


In [45]:
# matching the negation with 
negations = [
    (word("not"), ""),
    (word("isn't"), "is"),
    (word("cannot"), "can"),
    (word("can't"), "can"),
    (word("won't"), "will"),
    ("n't", ""), 
    ]

In [46]:
def match_rules(sentence, rules, defs):
    # match the sentences with the rules 
    sentence = clean(sentence)
    for rule in rules:
        result = match_rule(sentence, rule, defs)
        if result: 
            return result
    return match_literal(sentence, negations, defs)

In [47]:
def check(sentence, rule, defs):
    #check which rule is matched with the selected word and slicing the sentence into words
    output, rules = rule
    for pat in patterns:
        match = re.match(pat, sentence, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Applying recursion to the sentence to catch the matched word 
                groups[P] = match_rules(groups[P], rules, defs)[0]
            return '(' + output.format(**groups) + ')', defs

In [48]:
def match_literal(sentence, negations, defs):
    #if no word match or there is a negation, this block of code will handle the issues 
    polarity = ''
    for (neg, pos) in negations:
        (sentence, n) = re.subn(neg, pos, sentence, flags=re.I)
        polarity += n * '～'
    sentence = clean(sentence)
    P = proposition_name(sentence, defs)
    defs[P] = sentence
    return polarity + P, defs
    

In [62]:
# setting some symbols to be replaced with sentences 
def prop_symbols(sentence, defs, symbols='PQRSTF'):
    #return the symbols in the place of each selected word
    inverted = {defs[P]: P for P in defs}
    if sentence in inverted:
        return inverted[sentence]                      
    else:
        return next(P for P in symbols if P not in defs) 

In [63]:
def handle(text): 
    # remove the repeated space, comma, dots ...etc.
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')



In [67]:
match_rule("If you study hard, you will pass", logic('{P} ⇒ {Q}', 'if {P}, {Q}'),{})

('(P ⇒ Q)', {'P': 'you study hard', 'Q': 'you will pass'})

In [65]:
# some examples of propositional sentences
sentences = '''If it rains, then the ground is wet.
The sun is shining and the sky is blue.
Either it's Tuesday or it's Thursday.
If you study hard, you'll do well on the exam.
John is happy only if he has pizza.'''.split('.')


In [66]:
def rule(sentences, width=80): 
    #match the rules in each sentence of text to return the result
    for s in map(clean, sentences):
        logic, defs = match_rules(s, rules, {})
        print('\n' + textwrap.fill('Sentence: ' + s +'.', width), '\n\nLogic:', logic)
        for P in sorted(defs):
            print('{}: {}'.format(P, defs[P]))


In [61]:
#calling the function to get the result
rule(sentences)


Sentence: If it rains, then the ground is wet. 

Logic: (P ⇒ Q)
P: it rains
Q: the ground is wet

Sentence: The sun is shining and the sky is blue. 

Logic: (P ⋀ Q)
P: The sun is shining
Q: the sky is blue

Sentence: Either it's Tuesday or it's Thursday. 

Logic: (P ⋁ Q)
P: it's Tuesday
Q: it's Thursday

Sentence: If you study hard, you'll do well on the exam. 

Logic: (P ⇒ Q)
P: you study hard
Q: you'll do well on the exam

Sentence: John is happy only if he has pizza. 

Logic: (P ⇒ Q)
P: he has pizza
Q: John is happy only

Sentence: . 

Logic: P
P: 
